In [ ]:
import numpy as np
import pandas as pd

from tsfresh import extract_relevant_features
# from tsfresh.utilities.dataframe_functions import impute
# from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.distribution import MultiprocessingDistributor

labels = [6, 7]
features = {
    5: ['z_acc_clean', 'acceleration', 'z_gyro_clean', 'x_acc_clean', 'y_acc_clean', 'speed_kmh'],
    6: ['z_acc_clean', 'speed_kmh', 'z_gyro_clean', 'x_acc_clean', 'y_acc_clean'],
    7: ['z_acc_clean', 'speed_kmh', 'acceleration', 'z_gyro_clean', 'x_acc_clean', 'y_acc_clean']
}
basedir = 'D:/lalamove/lalamove/data/Clean_1s_all_240103/train'

In [ ]:
for label_title in labels:
    # load our data shape (sample_id, timestamp, time series)
    data_file = basedir + f'/{label_title}/chunk_1.npy'
    data = np.load(data_file)
    print('x', data.shape)

    label_file = basedir + f'/{label_title}/train_label_{label_title}.csv'
    label = pd.read_csv(label_file).label

    # pos = label[label == 1]
    # neg = label[label == 0].sample(frac=0.3, replace=False, random_state=123)
    # label_ = pd.concat((pos, neg)).reset_index()
    # data_ = data[label_.index]
    
    label_ = label
    data_ = data

    print('y', label, label.shape, label_.shape, data_.shape)

    # Create a list of column names for the DataFrame
    column_names = ['id', 'timestamp'] + features[label_title]

    # Reshape the numpy array to have 2 dimensions
    reshaped_array = data_.reshape(-1, data_.shape[2])

    # Create a pandas DataFrame
    my_df = pd.DataFrame(reshaped_array, columns=column_names[2:])
    print(my_df.shape)

    # Assign sample ids and timestamps to the DataFrame
    num_samples = data_.shape[0]
    num_timestamps = data_.shape[1]
    my_df['id'] = np.repeat(np.arange(num_samples), num_timestamps)
    my_df['timestamp'] = np.tile(np.arange(num_timestamps), num_samples)
    # my_df = my_df[column_names]

    # Print the resulting DataFrame
    print(my_df.shape)
    # print(my_df.head(4))

    Distributor = MultiprocessingDistributor(n_workers=4,
                                            disable_progressbar=False,
                                            progressbar_title="Feature Extraction")

    step = 10_000
    df_features = []
    i = 0
    for n in range(step, len(label), step):
        print(n)
        print(my_df[(n-step)*100:n*100].id.nunique())
        print(label[n-step:n].shape)
        try:
            # df_features.append(extract_relevant_features(my_df[(n-step)*100:n*100], label[n-step:n], column_id='id', column_sort='timestamp', column_kind=None, column_value=None))
            temp = extract_relevant_features(my_df[(n-step)*100:n*100], label[n-step:n], column_id='id', column_sort='timestamp', column_kind=None, column_value=None)
            temp.to_csv(basedir + f'/{label_title}/extract_features_{label_title}_{i}.csv')
            i += 1
        except:
            pass

    # df_features = []
    # df_features.append(extract_relevant_features(my_df[:10000*100], label[:10000], column_id='id', column_sort='timestamp', column_kind=None, column_value=None))

    # df_features_ = pd.concat(df_features)
    # print(df_features_.shape)
    # df_features_.to_csv(basedir + f'/{label_title}/extract_features_{label_title}.csv')